In [1]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

tokenizer = BertTokenizer.from_pretrained("../MARBERT_pytorch_verison/",local_files_only=True)
model = BertForMaskedLM.from_pretrained("../MARBERT_pytorch_verison/",local_files_only=True)


c:\Users\river\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at ../MARBERT_pytorch_verison/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
from transformers import pipeline
fill = pipeline('fill-mask', model=model, tokenizer=tokenizer)

In [2]:
def getLoss(true_sentence ,masked_sentence):
    inputs = tokenizer(masked_sentence, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits

    # retrieve index of [MASK]
    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    # print(logits.shape)
    predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
    # print(predicted_token_id)
    labels = tokenizer(true_sentence, return_tensors="pt")["input_ids"]
    # mask labels of non-[MASK] tokens
    labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

    outputs = model(**inputs, labels=labels)
    return (tokenizer.decode(predicted_token_id), round(outputs.loss.item(), 2))

In [3]:
import numpy as np
def getTopPred(masked_sentence,topN = 100):
    inputs = tokenizer(masked_sentence, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
    predicted_token_id = logits[0, mask_token_index]    
    topN_tokend_id = np.argpartition(predicted_token_id.reshape(-1), -topN)[-topN:]
    return tokenizer.decode(topN_tokend_id)

In [10]:
getTopPred("دا مجرد رأى و[MASK] اعلم و ابقا")


'الناس❉ لن اقدرالله ربما الدين هنا دا بعدين ربي لكى هى فيما النتيجة والله البادي ربى نا انتا المولى فعلا سبحانه لم بكره الذى يارب الباقى هو الواقع بالله ده هم انتم فالله اكيد رب كان تعالى ربنا بس وحده ربكم الرب الرحمن رينا اللى وربنا ريي انا انت الشارع الحق اللہ ربناا لكن اكاد اللهم ما لله هوا انى نعم سوف الرسول الموفق لا الشرع احنا ان الله بكرة غدا ﷲ اللھ url اللهه هوه ربك القادم االله فقط نحن المستقبل واللة كنت الحاضر هما العباد رسوله الخالق حضرتك الل التاريخ عايز م الفعل طبعا انتى اللة'

In [8]:
getLoss(true_sentence="دا مجرد رأى و الله اعلم و ابقا", masked_sentence="دا مجرد رأى و[MASK] اعلم و ابقا")

torch.Size([1, 10, 100000])
tensor([1944])


('الله', 0.11)

In [66]:
# دا مجرد رأى و الله اعلم و ابقا
fill("دا مجرد رأى و[MASK] اعلم و ابقا")

[{'score': 0.8920164108276367,
  'token': 1944,
  'token_str': 'ا ل ل ه',
  'sequence': 'دا مجرد راى و الله اعلم و ابقا'},
 {'score': 0.05682622268795967,
  'token': 2410,
  'token_str': 'ر ب ن ا',
  'sequence': 'دا مجرد راى و ربنا اعلم و ابقا'},
 {'score': 0.012048384174704552,
  'token': 2043,
  'token_str': 'ا ن ا',
  'sequence': 'دا مجرد راى و انا اعلم و ابقا'},
 {'score': 0.008924596942961216,
  'token': 2188,
  'token_str': 'و ا ل ل ه',
  'sequence': 'دا مجرد راى و والله اعلم و ابقا'},
 {'score': 0.0060321856290102005,
  'token': 16954,
  'token_str': 'ر ب ى',
  'sequence': 'دا مجرد راى و ربى اعلم و ابقا'}]

In [64]:
getLoss(true_sentence="هو كل يوم نسمع فتوى جديده  ماكلنا عارفين من كان مريضا أو على سفر وخلصت  هنخترع بقى رخص جديده من عندنا ", masked_sentence="هو كل يوم نسمع فتوى جديده  ماكلنا عارفين من كان [MASK] أو على سفر وخلصت  هنخترع بقى رخص جديده من عندنا ")

('مسافر', 6.15)

## test strategy

In [106]:
"المشتروع" in tokenizer.vocab.keys()

False

In [86]:
# انا رايح اركب المشتروع
fill("انا رايح اركب [MASK]")

[{'score': 0.07754268497228622,
  'token': 12264,
  'token_str': 'ع ج ل',
  'sequence': 'انا رايح اركب عجل'},
 {'score': 0.052763354033231735,
  'token': 33856,
  'token_str': 'ت ا ك س ي',
  'sequence': 'انا رايح اركب تاكسي'},
 {'score': 0.04821640998125076,
  'token': 30165,
  'token_str': 'ا ل ق ط ر',
  'sequence': 'انا رايح اركب القطر'},
 {'score': 0.04384848475456238,
  'token': 60969,
  'token_str': 'ت و ك ت و ك',
  'sequence': 'انا رايح اركب توكتوك'},
 {'score': 0.04264984279870987,
  'token': 1936,
  'token_str': 'u r l',
  'sequence': 'انا رايح اركب url'}]

In [62]:
pred_top_100 = getTopPred("انا رايح اركب [MASK]")
pred_top_100

'اوتوستراد الهرم تاني قارب البوكيمون جنازة مواصلة فيراري موتو صاروخ واحد السيسي كفر دلوقتي القمر. السيارة الماتش المشروع اليكس التوكتوك مراجيح ترام حديد طيارات البحر ورا الصحراوي السياره بنزين الموج تقويم عربية عربيه بوكيمون الاسعاف سواق الزمالك مركب حافلة الجيش لوحدي مواصله ايه سفينة الشنطه الدايري [UNK] حشيش مكنة المحور تكسي بترول bmw الخيل الحصان حصان المرسيدس اهو الصاروخ المواصلات الهوا حمار العجله الميكروباص اتوبيس عجلة خيل ترحال مترو العربية طياره مرسيدس الموجه عجل مواصلات الطياره url القطر كريم الموجة العجلة العربيه طيارة قطار قطر الطيارة توكتوك اوبر موجة ميكروباص المترو الباص تاكسي باص جمل الاتوبيس القطار التاكسي عجله'

In [65]:
"المشروع" in tokenizer.vocab.keys()

True

In [63]:
"المشروع" in pred_top_100

True

In [ ]:
point = 1

In [64]:
"المكتب" in pred_top_100

False

In [222]:
(predicted, loss_actual) = getLoss(true_sentence="انا رايح اركب المشروع", masked_sentence="انا رايح اركب [MASK]")
(predicted, loss_actual) 

('عجل', 6.78)

In [223]:
(predicted, loss_relevant) = getLoss(true_sentence="انا رايح اركب عجل", masked_sentence="انا رايح اركب [MASK]")
(predicted, loss_relevant) 

('عجل', 2.56)

In [224]:
(predicted, loss_not_relevant) = getLoss(true_sentence="انا رايح اركب المكتب", masked_sentence="انا رايح اركب [MASK]")
(predicted, loss_not_relevant) 

('عجل', 10.85)

In [225]:
# add award point to relevant distance because it appeared in the top 100 predicted words 
point = .5 * loss_actual 

loss_actual = loss_actual- point
loss_not_relevant = loss_not_relevant + point

actual_relevant_dist = abs( loss_actual - loss_relevant  )
actual_not_relevant_dist = abs( loss_actual - loss_not_relevant )

total_distance  = actual_relevant_dist + actual_not_relevant_dist

actual_relevant_precent = actual_not_relevant_dist  / total_distance
actual_notrelevant_precent = actual_relevant_dist / total_distance

print(f"Percentage of Relevance {round(actual_relevant_precent * 100 , 2)} %" )
print(f"Percentage of Not Relevant {round(actual_notrelevant_precent * 100 , 2)} %")

Percentage of Relevance 92.89 %
Percentage of Not Relevant 7.11 %


In [187]:
actual_relevant_precent + actual_notrelevant_precent

1.0

## TEST 2

In [13]:
# After correcting "نرم" >> "نجم"
true_sentence = "بقينا ف زمن اللي سب الدين مش ف لسانه ده نجم"
masked_sentence = "بقينا ف زمن اللي سب الدين مش ف لسانه ده [MASK]"

In [89]:
"نرم" in tokenizer.vocab.keys()

False

In [91]:
"نجم" in tokenizer.vocab.keys()

True

In [92]:
"نجم" in pred_top_100

False

In [14]:
pred_top_100 = getTopPred(masked_sentence)
pred_top_100

"😏😏 دلوقتي ارحمونا تاني♨ 💔 اخلاقنا 😑😑 - ارهاب دينه 👌🏻 ابتلاء ☺ تويتر غلط اخلاق جهل 😅 😡 😓 😀 خلاص 😞 عادي مجتمع🌟 النفاق 😠 😊☁ مصيبه كمان انتشر اسلوب كله ستايل ✋✋ قرف. موضه اخره الواقع الطبيعي مرض حقيقه بس اسمه والله لعنه6 👎 ✋ 😏 👌 هبل 👊 خالص 😐 نفاق للاسف بقي هزار 😑 حوار كلام 😃 بجد حقيقي 😂 تخلف ؟ الزمن 😒 بقى٨ عيب واقع'حلال كافر 😔 فرعون حقيقة 😕 😂😂 حالنا مجتمعنا [UNK] ، url! حرام حاله زمان زمن بقا كفر ✋🏻 😒😒"

In [105]:
point = 0

In [94]:
"المكتب" in pred_top_100

False

In [ ]:
fill(masked_sentence)

[{'score': 0.1302552968263626,
  'token': 16,
  'token_str': '.',
  'sequence': 'بقينا ف زمن اللي سب الدين مش ف لسانه ده.'},
 {'score': 0.11558032035827637,
  'token': 5,
  'token_str': '!',
  'sequence': 'بقينا ف زمن اللي سب الدين مش ف لسانه ده!'},
 {'score': 0.07560965418815613,
  'token': 1936,
  'token_str': 'u r l',
  'sequence': 'بقينا ف زمن اللي سب الدين مش ف لسانه ده url'},
 {'score': 0.042377058416604996,
  'token': 372,
  'token_str': '✋',
  'sequence': 'بقينا ف زمن اللي سب الدين مش ف لسانه ده ✋'},
 {'score': 0.020195644348859787,
  'token': 906,
  'token_str': '😒',
  'sequence': 'بقينا ف زمن اللي سب الدين مش ف لسانه ده 😒'}]

In [226]:
(predicted, loss_actual) = getLoss(true_sentence="بقينا ف زمن اللي سب الدين مش ف لسانه ده نجم", masked_sentence= masked_sentence)
(predicted, loss_actual) 

('.', 11.2)

In [227]:
(predicted, loss_relevant) = getLoss(true_sentence=f"بقينا ف زمن اللي سب الدين مش ف لسانه ده {predicted}", masked_sentence=masked_sentence)
(predicted, loss_relevant) 

('.', 2.04)

In [228]:
(predicted, loss_not_relevant) = getLoss(true_sentence="بقينا ف زمن اللي سب الدين مش ف لسانه ده المكتب", masked_sentence=masked_sentence)
(predicted, loss_not_relevant) 

('.', 16.7)

In [229]:
loss_not_relevant

16.7

In [230]:
# add penality point to relevant distance because it didn't appeared in the top 100 predicted words 
point = .5 * loss_actual 

loss_actual = loss_actual + point
loss_not_relevant = loss_not_relevant - point

actual_relevant_dist = abs( loss_actual - loss_relevant  )
actual_not_relevant_dist = abs( loss_actual - loss_not_relevant )

total_distance  = actual_relevant_dist + actual_not_relevant_dist

actual_relevant_precent = actual_not_relevant_dist  / total_distance
actual_notrelevant_precent = actual_relevant_dist / total_distance

print(f"Percentage of Relevance {round(actual_relevant_precent * 100 , 2)} %" )
print(f"Percentage of Not Relevant {round(actual_notrelevant_precent * 100 , 2)} %")

Percentage of Relevance 27.86 %
Percentage of Not Relevant 72.14 %


In [110]:
abs( loss_actual - loss_not_relevant  )

5.5

In [111]:
abs( loss_actual - loss_relevant  )


9.16

In [108]:
total_distancs

14.66

In [26]:
type(tokenizer.vocab.keys()- pred_top_100.split())

set

In [23]:
tokenizer.tokenize("##ﻈﺮ")

['[UNK]', '[UNK]', 'ﻈ', '##ﺮ']

# Function

In [24]:
import random
def getRelevanceScore(true_sentence, word):
    masked_sentence = true_sentence.replace(word,"[MASK]")
    pred_top_100 = getTopPred(masked_sentence)
    irrelevant_word = random.choices(list(tokenizer.vocab.keys()- pred_top_100.split()), k=1)[0]
    print("irrelevant_word in pred_top_100: ",irrelevant_word, irrelevant_word in pred_top_100)
    (predicted, loss_actual) = getLoss(true_sentence, masked_sentence)
    (predicted, loss_relevant) = getLoss(true_sentence.replace(word, predicted), masked_sentence)
    n_tokens = len(tokenizer.tokenize(irrelevant_word))
    print("n_tokens: ",n_tokens)
    (predicted, loss_not_relevant) = getLoss(true_sentence.replace(word, irrelevant_word), masked_sentence = true_sentence.replace(word, " ".join(["[MASK]"]*n_tokens)))
    
    print("predicted", predicted)
    print("loss_actual", loss_actual)
    print("loss_relevant", loss_relevant)
    print("loss_not_relevant", loss_not_relevant)
    
    point = .5 * loss_actual 
    

    if word not in pred_top_100:
        point = -1* point
        print("Apply penality  = ", point)
    else:
        print("Apply reward  = ", point)
        
    loss_actual = loss_actual- point
    loss_not_relevant = loss_not_relevant + point
    
    actual_relevant_dist = abs( loss_actual - loss_relevant  )
    actual_not_relevant_dist = abs( loss_actual - loss_not_relevant )

    total_distance  = actual_relevant_dist + actual_not_relevant_dist

    actual_relevant_precent = actual_not_relevant_dist  / total_distance
    actual_notrelevant_precent = actual_relevant_dist / total_distance

    print(f"Percentage of Relevance {round(actual_relevant_precent * 100 , 2)} %" )
    print(f"Percentage of Not Relevant {round(actual_notrelevant_precent * 100 , 2)} %")
    return (actual_relevant_precent, actual_notrelevant_precent)


In [25]:

getRelevanceScore("بقينا ف زمن اللي سب الدين مش ف لسانه ده نجم","نجم")

irrelevant_word in pred_top_100:  مبيبقاش False
n_tokens:  1
predicted .
loss_actual 11.2
loss_relevant 2.04
loss_not_relevant 16.65
Apply penality  =  -5.6
Percentage of Relevance 28.04 %
Percentage of Not Relevant 71.96 %


(0.2803510482691369, 0.719648951730863)

In [26]:
getRelevanceScore("انا رايح اركب المشروع","المشروع")

irrelevant_word in pred_top_100:  فاشغله False
n_tokens:  1
predicted عجل
loss_actual 6.78
loss_relevant 2.56
loss_not_relevant 19.68
Apply reward  =  3.39
Percentage of Relevance 95.95 %
Percentage of Not Relevant 4.05 %


(0.9595319356411507, 0.04046806435884935)